In [25]:
from datetime import datetime
import planet
from planet import Auth
from planet import Session, data_filter
import json
import time
import os
import asyncio

# Helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

In [5]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = ''
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

In [8]:
#Path to fissure study area GeoJSONs 
fissures_path="AZGS_shapefiles/gisdata/shapefiles/"

study_area=fissures_path+'study_area_epsg4326.geojson'

#Load study areas
with open(study_area, 'r') as f:
    geojson_data = json.load(f)

In [10]:
#The third coordinate in the study area GeoJSONs needs to be removed to be read by the geometry filter, here's a function to that:
def remove_third_element(coords):
    return [coord[:2] for coord in coords]

In [11]:
# Apply this to all coordinates in the GeoJSON
for feature in geojson_data['features']:
    if feature['geometry']['type'] == 'Polygon':
        feature['geometry']['coordinates'] = [remove_third_element(polygon) for polygon in feature['geometry']['coordinates']]
    elif feature['geometry']['type'] == 'MultiPolygon':
        feature['geometry']['coordinates'] = [[remove_third_element(polygon) for polygon in multipolygon] for multipolygon in feature['geometry']['coordinates']]

geometries = [feature['geometry'] for feature in geojson_data['features']]

In [12]:
feature['geometry']

{'type': 'MultiPolygon',
 'coordinates': [[[[-112.126724075808, 33.04232301812071],
    [-112.2405953979192, 33.04124254762971],
    [-112.2414944364583, 33.10667881656496],
    [-112.1791220177617, 33.10728457153469],
    [-112.1471345511401, 33.1348052168475],
    [-112.1279065652451, 33.13498079831125],
    [-112.1275586350043, 33.10776177632491],
    [-112.126724075808, 33.04232301812071]]]]}

In [19]:
#Possible item types
planet.specs.get_item_types()

{'Landsat8L1G',
 'MOD09GA',
 'MOD09GQ',
 'MYD09GA',
 'MYD09GQ',
 'PSOrthoTile',
 'PSScene',
 'REOrthoTile',
 'REScene',
 'Sentinel1',
 'Sentinel2L1C',
 'SkySatCollect',
 'SkySatScene'}

In [27]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]# look up SkySat code, make separate search for all

#Geometry filter
geom_filters = [data_filter.geometry_filter(geom) for geom in geometries]

#Percentage clear pixel filter - not super sure how that's different to cloud cover?
clear_percent_filter = data_filter.range_filter('clear_percent', 95)

#date filter
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=12, day=1, year=2023),lte = datetime(month=1, day=1, year=2024))

#cloud cover filter
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.05)

#Combine the filters into one filter
combined_filters = [data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter]) for geom_filter in geom_filters]

In [28]:
# Build the requests asynchronously
async with Session() as sess:
    cl = sess.client('data')
    requests = [await cl.create_search(name='dec_23_all_fissure_areas',search_filter=combined_filter, item_types=item_types) for combined_filter in combined_filters]

In [29]:
len(requests)

26

In [30]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our result limit to 50.
async with Session() as sess:
    item_lists=[]
    for a in requests:
        cl = sess.client('data')
        request = a
        items = cl.run_search(search_id=request['id'], limit=50)
        item_list = [i async for i in items]
        item_lists.append(item_list)

In [264]:
# Save JSONs with search results if wanted
for a in range(0,len(item_lists)):
    with open(f'output/results_itemlist{a}.json','w') as f:
        jsonStr = json.dumps(item_list)
        f.write(jsonStr)
        f.close()

In [ ]:
#Ok, the tricky bit. Define functions to activate asset (or return an exception message) and to dowload the asses if it is active (or return 1)error message that asset isn't activated yet or 2)exception message indicating error in downloading asset. 

async def activate_asset(cl, item_type, item_id, asset_type, directory):
    try:
        # Get asset
        asset_desc = await cl.get_asset(item_type_id=item_type, item_id=item_id, asset_type_id=asset_type)
        # Activate asset
        await cl.activate_asset(asset=asset_desc)
        return (asset_desc, directory)
    except Exception as e:
        print(f"Error in activating asset: {e}")
        return None

async def download_asset(cl, asset_desc, directory):
    try:
        # Wait for asset
        await cl.wait_asset(asset=asset_desc)
        # Download asset
        asset_path = await cl.download_asset(asset=asset_desc, directory=directory, overwrite=True)
        print(f"Downloaded {asset_path}")
    except KeyError:
        print("Asset not active yet.")
    except Exception as e:
        print(f"Error in downloading asset: {e}")

        
async def main():
    async with Session() as sess:

        # Try to activate everything!
        activation_tasks = []
        for item_list in item_lists:
            for item in item_list:
                item_id = item['id']
                item_type = item['properties']['item_type']
                cl = sess.client('data')

                # Define output directories
                output_8b = 'output/ortho_analytic_8b'
                output_8b_xml = 'output/ortho_analytic_8b_xml'
                output_visual = 'output/ortho_visual'

                # Start activating assets and store tasks
                activation_tasks.append(activate_asset(cl, item_type, item_id, 'ortho_analytic_8b', output_8b))
                activation_tasks.append(activate_asset(cl, item_type, item_id, 'ortho_analytic_8b_xml', output_8b_xml))
                activation_tasks.append(activate_asset(cl, item_type, item_id, 'ortho_visual', output_visual))

        # Wait for all activations to complete
        activated_assets = await asyncio.gather(*activation_tasks)

        # Download assets
        for asset_info in activated_assets:
            if asset_info:
                asset_desc, directory = asset_info
                await download_asset(cl, asset_desc, directory)

await main()

output/ortho_analytic_8b/20231231_172605_15_2458_3B_AnalyticMS_8b.tif: 100%|██████████| 856k/856k [00:14<00:00, 59.9MB/s] 


Downloaded output/ortho_analytic_8b/20231231_172605_15_2458_3B_AnalyticMS_8b.tif


output/ortho_analytic_8b_xml/20231231_172605_15_2458_3B_AnalyticMS_8b_metadata.xml: 100%|██████████| 0.01k/0.01k [00:00<00:00, 1.64MB/s]


Downloaded output/ortho_analytic_8b_xml/20231231_172605_15_2458_3B_AnalyticMS_8b_metadata.xml


output/ortho_visual/20231231_172605_15_2458_3B_Visual.tif: 100%|██████████| 180k/180k [00:04<00:00, 43.3MB/s] 


...

output/ortho_analytic_8b/20231212_180458_64_247d_3B_AnalyticMS_8b.tif: 100%|██████████| 955k/955k [00:16<00:00, 60.2MB/s] 


Downloaded output/ortho_analytic_8b/20231212_180458_64_247d_3B_AnalyticMS_8b.tif


output/ortho_analytic_8b_xml/20231212_180458_64_247d_3B_AnalyticMS_8b_metadata.xml: 100%|██████████| 0.01k/0.01k [00:00<00:00, 1.55MB/s]


Downloaded output/ortho_analytic_8b_xml/20231212_180458_64_247d_3B_AnalyticMS_8b_metadata.xml


output/ortho_visual/20231212_180458_64_247d_3B_Visual.tif:  52%|█████▏    | 114k/218k [00:01<00:01, 67.8MB/s] 